In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from rpy2.robjects.packages import importr

utils = importr("utils")
utils.chooseCRANmirror(ind=1)
utils.install_packages('lme4')

%load_ext rpy2.ipython

In [ ]:
# Load data
utterances = pd.read_csv("results/intelligibility/conversations.csv")

# convert True/False to 0/1:
utterances.replace({False: 0, True: 1}, inplace=True)

utterances.head()


In [ ]:
# Some children have very few data points:
counts = utterances.groupby("child_name").size()
counts

In [ ]:
# Exclude children with less than 100 datapoints:
# child_names_enough_data = [name for name, count in counts.items() if count > 100]
# print(len(utterances))
# utterances = utterances[utterances.child_name.isin(child_names_enough_data)]
# print(len(utterances))

## Caregiver Contingency
### By child

In [ ]:
plt.rcParams['figure.figsize'] = [15, 10]
plt.figure()
sns.barplot(
        data=utterances,
        x="child_name",
        y="has_response",
        hue="is_intelligible"
)
plt.title("Caregiver contingency - by child")
plt.xticks(rotation=75)
plt.subplots_adjust(bottom=0.3)

### By Age

In [ ]:
plt.rcParams['figure.figsize'] = [15, 10]
sns.barplot(
        data=utterances,
        x="age",
        y="has_response",
        hue="is_intelligible"
)
plt.title("Caregiver contingency - by age")

## Child contingency
### By Child


In [ ]:
plt.rcParams['figure.figsize'] = [15, 10]
plt.figure()
sns.barplot(
        data=utterances[utterances["is_intelligible"] == 1],
        x="child_name",
        y="follow_up_intelligible",
        hue="has_response"
)
plt.title("Child contingency - by child")
plt.xticks(rotation=75)
plt.subplots_adjust(bottom=0.3)

### By Age

In [ ]:
plt.rcParams['figure.figsize'] = [15, 10]
sns.barplot(
        data=utterances[utterances["is_intelligible"] == 1],
        x="age",
        y="follow_up_intelligible",
        hue="has_response"
)
plt.title("Child contingency - by age")

## Caregiver contingency
### GLM (with mixed effects)

In [ ]:
# normalize age
min_age, max_age = utterances["age"].min(), utterances["age"].max()
utterances["age"] = (utterances["age"] - min_age) / (max_age - min_age) * (1 - 0)


In [ ]:
%%R -i utterances
library(lme4)

# Caregiver contingency
m_caregiver_contingency<-glmer('has_response ~ is_intelligible * age + (1 | child_name)', data=utterances, family=binomial) #, control=glmerControl(optimizer="bobyqa",optCtrl=list(maxfun=2e5)))
print(summary(m_caregiver_contingency))


## Child Contingency
### GLM (with mixed effects)

In [ ]:
%%R -i utterances
library(lme4)

utterances_child_intelligible = subset(utterances, is_intelligible==1)

# Child contingency
m_child_contingency<-glmer('follow_up_intelligible ~ has_response * age + (1 | child_name)', data=utterances_child_intelligible, family=binomial)
print(summary(m_child_contingency))